In [1]:
#Start sesion Sparks

from pyspark.sql import SparkSession
from pyspark import SparkContext
spark = SparkSession.builder \
  .appName('fact_tables_magdiel') \
  .config('spark.jars', 'gs://spark-lib/bigquery/spark-bigquery-latest_2.12.jar') \
  .getOrCreate()

spark.conf.set("spark.sql.repl.eagerEval.enabled",True)

In [ ]:
#name table compras
table_compras = "becade_mgutierrez.pr_compras"

#load table
raw_compras = spark.read \
  .format("bigquery") \
  .option("table", table_compras) \
  .load()

#show schema
raw_compras.printSchema()

#show incoming lines
print("lines incoming: " , raw_compras.count())

In [7]:
#name table compras
table_rate = "becade_mgutierrez.pr_product_avg_rate"

#load table
raw_rate = spark.read \
  .format("bigquery") \
  .option("table", table_rate) \
  .load()

#show schema
raw_rate.printSchema()

#show incoming lines
print("lines incoming: " , raw_rate.count())

root
 |-- product_id: string (nullable = true)
 |-- product_avg_rate: double (nullable = true)
 |-- country_count: long (nullable = false)

lines incoming:  8246


In [8]:
raw_rate.show(10)

+----------+----------------+-------------+
|product_id|product_avg_rate|country_count|
+----------+----------------+-------------+
|B09P7X5R6T|             2.0|            1|
|B095SDZM5N|             2.0|            1|
|B093CQY35F|             2.0|            1|
|B09BV73GLJ|             2.0|            1|
|B088B8MCCF|             2.0|            1|
|B096ZRH8W7|             2.0|            1|
|B09GK9XDWR|             2.5|            1|
|B08NXYNSLX|             2.5|            1|
|B09LS574M5|             2.5|            1|
|B09PYH6PQV|             2.5|            1|
+----------+----------------+-------------+
only showing top 10 rows



In [18]:
raw_compras.filter(raw_compras.product_id == "B09FCXXGT5").show(truncate=False)

+-----------------+-------------+----------+----------------+-------------+
|client_id        |product_price|product_id|product_quantity|purchase_date|
+-----------------+-------------+----------+----------------+-------------+
|209-696678-32-117|59.76        |B09FCXXGT5|2               |2011-12-24   |
|209-696678-32-117|59.76        |B09FCXXGT5|2               |2012-04-20   |
|209-696678-32-117|59.76        |B09FCXXGT5|2               |2012-05-13   |
|209-696678-32-117|59.76        |B09FCXXGT5|2               |2012-06-09   |
|209-696678-32-117|59.76        |B09FCXXGT5|2               |2012-12-09   |
|209-696678-32-117|59.76        |B09FCXXGT5|2               |2013-01-05   |
|209-696678-32-117|59.76        |B09FCXXGT5|2               |2015-06-25   |
|209-696678-32-117|59.76        |B09FCXXGT5|2               |2016-04-27   |
|209-931475-61-720|59.76        |B09FCXXGT5|2               |2015-08-09   |
|209-931475-61-720|59.76        |B09FCXXGT5|2               |2016-06-17   |
|209-931475-

In [10]:
##Obtener AÑO



In [11]:
from pyspark.sql.functions  import year, month

df_fact_raw= raw_compras.withColumn('purchase_year',year(raw_compras.purchase_date)) 
df_fact_raw.show(5)
#df_new_sales.printSchema()
print("lines source: " , df_fact_raw.count())

+-----------------+-------------+----------+----------------+-------------+-------------+
|        client_id|product_price|product_id|product_quantity|purchase_date|purchase_year|
+-----------------+-------------+----------+----------------+-------------+-------------+
|209-696678-32-117|       473.98|B00N69D6AS|               2|   2012-03-16|         2012|
|209-696678-32-117|       473.98|B00N69D6AS|               2|   2013-03-22|         2013|
|209-696678-32-117|       473.98|B00N69D6AS|               2|   2014-03-02|         2014|
|209-696678-32-117|       473.98|B00N69D6AS|               2|   2016-01-03|         2016|
|213-216004-61-320|       473.98|B00N69D6AS|               2|   2013-06-09|         2013|
+-----------------+-------------+----------+----------------+-------------+-------------+
only showing top 5 rows

lines source:  1965807


In [12]:
#Group by

In [24]:
from pyspark.sql.functions  import countDistinct,sum,count

df_fact_sales = df_fact_raw.select('*') \
                .groupBy('purchase_year','product_id') \
                .agg(sum('product_price').alias('purchase_sales'),
                     countDistinct('client_id').alias('client_quantity'), 
                     sum('product_quantity').alias('product_quantity_sales') ) \
                .sort(['purchase_year', 'product_id'], ascending=False)
                 
print("lines source: " , df_fact_sales.count())

df_fact_sales.show(10)

lines source:  781
+-------------+----------+------------------+---------------+----------------------+
|purchase_year|product_id|    purchase_sales|client_quantity|product_quantity_sales|
+-------------+----------+------------------+---------------+----------------------+
|         2020|B09P5BBPVY| 265454.2799999998|            100|                  5772|
|         2020|B09FM7CKG1| 56771.82000000001|            100|                  1959|
|         2020|B09FCXXGT5|112020.12000000007|            100|                  3749|
|         2020|B09BNK4592|111496.56000000007|            100|                  1862|
|         2020|B098RKWHHZ| 695430.1299999999|            100|                  1987|
|         2020|B098P1M628| 551117.1599999984|            100|                 11484|
|         2020|B097Y38X79| 537275.8800000008|            100|                 11532|
|         2020|B0975P2RBR| 79900.01999999999|            100|                  1998|
|         2020|B0931NN4PR|          195552.0| 

In [28]:
from pyspark.sql.functions  import col
 
#InnerJoin df_fact_sales &&  raw_rate
full_table_fact = df_fact_sales.alias('A').join(raw_rate.alias('B'), \
                (col('A.product_id') == col('B.product_id')) , "inner") 

#Show first 20 rows
full_table_fact= full_table_fact.select('A.purchase_year','A.product_id','A.purchase_sales','A.client_quantity','A.product_quantity_sales','B.product_avg_rate')
                 

print("lines source: " , full_table_fact.count())
full_table_fact.show(13)

lines source:  770
+-------------+----------+------------------+---------------+----------------------+----------------+
|purchase_year|product_id|    purchase_sales|client_quantity|product_quantity_sales|product_avg_rate|
+-------------+----------+------------------+---------------+----------------------+----------------+
|         2020|B09FM7CKG1| 56771.82000000001|            100|                  1959|             3.6|
|         2020|B09FCXXGT5|112020.12000000007|            100|                  3749|             4.8|
|         2020|B09BNK4592|111496.56000000007|            100|                  1862|             4.7|
|         2020|B098RKWHHZ| 695430.1299999999|            100|                  1987|             4.8|
|         2020|B098P1M628| 551117.1599999984|            100|                 11484|             3.9|
|         2020|B097Y38X79| 537275.8800000008|            100|                 11532|             3.9|
|         2020|B0975P2RBR| 79900.01999999999|            100|  